In [ ]:
# Install libraries if necessary
!pip install lightgbm
!pip install matplotlib
!pip install numpy
!pip install pandas
!pip install IPython

In [ ]:
!conda install -c anaconda pygraphviz --yes

In [ ]:
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pygraphviz
from IPython.display import display, Image

In [ ]:
# Load data from my work github
# github_url = 'https://raw.github.expedia.biz/landrejek/flight_ranking_demo/main/df_flights.csv?token=GHSAT0AAAAAAAAATMRA4RPOH6IYWLPGLXREZIGC2EQ'
# Load data from my personal github
github_url = 'https://raw.githubusercontent.com/LukeAndrejek/flight_ranking_demo/main/df_flights.csv'

df_flights = pd.read_csv(github_url)
df_flights['carrier'] = df_flights['carrier'].astype('category')
df_flights.tail()

In [ ]:
# View the bookings of a particular user
example_user_id = 336
df_flights[(df_flights['user_id'] == example_user_id) & (df_flights['booked'] == 1)].sort_values(by=['day'])

In [ ]:
# Prepare training data
model_features = ['price', 'duration', 'layover_hours', 'legs', 'carrier', 'departure_hour', 'arrival_hour', 'recent_bookings_with_carrier']
df_train = df_flights[df_flights['day'] > 7]
X_train = df_train[model_features]
y_train = df_train['booked']
group_sizes_train = df_train.groupby('query_id', sort=False)['query_id'].count().to_numpy()

In [ ]:
# Build model
model = (
    lgb.LGBMRanker(objective = 'lambdarank',
                   num_leaves = 10,
                   seed = 42)
    .fit(X=X_train,
         y=y_train,
         group=group_sizes_train)
)

In [ ]:
# Plot feature importances
lgb.plot_importance(model)

In [ ]:
# Plot impact of a particular feature on model score
plot_offer_row_index = 0
# Recall model_features = ['price', 'duration', 'layover_hours', 'legs', 'carrier', 'departure_hour', 'arrival_hour', 'recent_bookings_with_carrier']
plot_feature_index = 7

assert(df_train[model_features[plot_feature_index]].dtype != 'category')  # This code only plots numeric features

# Prepare feature data
model_features = model.feature_name_
plot_offer_row = X_train.iloc[[plot_offer_row_index]]
plot_feature =  model_features[plot_feature_index]
domain = np.linspace(0,10,20)

# Plot result
plt.plot(domain,[model.predict(plot_offer_row.assign(**{plot_feature:x})) for x in domain])
plt.xlabel(plot_feature)
plt.ylabel('model score')
plt.grid(True)

In [ ]:
# Plot a particular tree
tree_index = 0  # Set value to 8 to view a tree featuring recent_bookings_with_carrier

model_digraph = lgb.create_tree_digraph(model, tree_index=tree_index)
model_digraph.format = 'png'
png_path = model_digraph.render(filename='model_digraph')
display(Image(png_path))